# HiFive Unmatched Power Monitor Data Viewer

In [ ]:
import datetime
import nptdms
import numpy
import pandas
import pathlib

from bokeh.models import Legend
from bokeh.palettes import Category20
from bokeh.plotting import figure
from bokeh.plotting import output_notebook
from bokeh.plotting import show

### Functions

In [ ]:
def load_measurements(
        tdms_path,
        tdms_group_name):
    """Load current measurements from a input TDMS file.
    
    Load current measurements and channel metadata from the input TDMS
    file, then compute power traces.
    """
    with nptdms.TdmsFile.open(tdms_path) as tdms_file:
        # Check the expected measurement group is present in the
        # input TDMS file.
        found = False
        for group in tdms_file.groups():
            if tdms_group_name == group.name:
                found = True
        if not found:
            raise ValueError(f"TDMS group `{tdms_group_name}` not found")

        # Load current measurements.
        current = tdms_file[tdms_group_name].as_dataframe(
            time_index=True,
            absolute_time=False
        )
            
        # Load channel metadata.
        meta = {}
        for channel in tdms_file[tdms_group_name].channels():
            for key, value in channel.properties.items():
                if key not in meta:
                    meta[key] = []
                meta[key].append(value)
        meta = pandas.DataFrame(meta)

    return current, meta


def compute_power_consumption(
        current,
        pin_vdd):
    """Compute power traces from current consumption.
    
    Use current measurements and power line voltage to compute
    power consumption for each line.
    """
    # Compute power traces.
    power = pandas.DataFrame(
        data={
            pin: current[pin] * vdd
            for pin, vdd in pin_vdd.items()
            if pin in current.columns
        },
        index=current.index
    )
    
    # Compute overall power consumption by line.
    overall = power.mean()
    
    # Normalize power line contribution.
    normalized = overall / overall.sum()
    
    return power, overall, normalized


def average_consumption(
        power,
        time_interval):
    """Average power consumption over a fixed-size time window.
    """
    # Compute number of time windows of the specified length in
    # the power trace.
    n_intervals = numpy.uint(power.index.max() / time_interval)
    
    # Compute average power consumption for each time interval.
    average = pandas.DataFrame(
        data=[
            power[i*time_interval:(i+1)*time_interval].mean()
            for i in range(n_intervals)
        ],
        index=[
            i * time_interval
            for i in range(n_intervals)
        ]
    )
    
    return average


def make_plot_figure(
        plot_width,
        plot_height,
        plot_xlabel,
        plot_ylabel):
    """Create a figure to plot data into.
    """
    # Create figure.
    fig = figure(
        plot_width=plot_width,
        plot_height=plot_height,
        tools="box_zoom,save,reset",
        toolbar_location="below"
    )
    
    # Configure axis labels.
    fig.xaxis.axis_label = plot_xlabel
    fig.xaxis.axis_label_text_font = "Arial"
    fig.xaxis.axis_label_text_color = "black"
    fig.xaxis.axis_label_text_font_size = "11pt"
    fig.xaxis.axis_label_text_font_style = "bold"
    fig.yaxis.axis_label = plot_ylabel
    fig.yaxis.axis_label_text_font = "Arial"
    fig.yaxis.axis_label_text_color = "black"
    fig.yaxis.axis_label_text_font_size = "11pt"
    fig.yaxis.axis_label_text_font_style = "bold"
    
    # Configure major labels.
    fig.xaxis.major_label_text_color = "black"
    fig.xaxis.major_label_text_font = "Arial"
    fig.xaxis.major_label_text_font_size = "11pt"
    fig.yaxis.major_label_text_color = "black"
    fig.yaxis.major_label_text_font = "Arial"
    fig.yaxis.major_label_text_font_size = "11pt"

    # Create legend.
    legend = Legend(
        label_text_font="Arial",
        label_text_font_size="9pt",
        label_text_color="black",
        border_line_color="black",
        border_line_alpha=1,
        border_line_width=1,
        click_policy="hide"
    )
    fig.add_layout(legend, "right")
    
    return fig


def plot_power_signals(
        traces,
        plot_width=950,
        plot_height=350):
    """Plot all power traces in the same figure.
    """
    fig =  make_plot_figure(
        plot_width,
        plot_height,
        "Time [s]",
        "Power [W]"
    )
    palette = Category20[traces.columns.size]
    for i, column in enumerate(traces.columns):
        fig.line(
            "index",
            column,
            source=traces,
            color=palette[i],
            legend_label=column,
            line_width=1.5
        )

    return fig


def plot_power_contributions_vbar(
        power,
        plot_width=950,
        plot_height=350):
    """
    """
    fig =  make_plot_figure(
        plot_width,
        plot_height,
        "Time [s]",
        "Power [W]"
    )
    palette = Category20[power.columns.size]
    fig.vbar_stack(
        power.columns.to_list(),
        x="index",
        source=power,
        color=palette,
        width=(power.index[1] - power.index[0]) * 0.8,
        legend_label=power.columns.to_list(),
        line_color="black",
        fill_alpha=0.75
    )
    
    return fig


def plot_power_contributions_varea(
        power,
        plot_width=950,
        plot_height=350):
    """
    """
    fig =  make_plot_figure(
        plot_width,
        plot_height,
        "Time [s]",
        "Power [W]"
    )
    palette = Category20[power.columns.size]
    fig.varea_stack(
        power.columns.to_list(),
        x='index',
        color=palette,
        legend_label=power.columns.to_list(),
        source=power,
        fill_alpha=0.75
    )

    return fig

### Constants

In [ ]:
POWER_LINE_PIN_VDD = {
    "i_core":    0.918956,
    "i_ddr_soc": 1.197213,
    "i_io":      1.793037,
    "i_pll":     1.794147,
    "i_pcievp":  0.923521,
    "i_pcievph": 1.792846,
    "i_ddr_mem": 1.197322,
    "i_ddr_pll": 0.919675,
    "i_ddr_vpp": 2.442895
}
POWER_AVG_WINDOW_COARSE = 100e-3
POWER_AVG_WINDOW_FINE = 100e-6

In [ ]:
# Ask the user to input a directory path where to look for TDMS files.
BASE_DIR_PATH = input("TDMS files directory path: ")
BASE_DIR_PATH = pathlib.Path(BASE_DIR_PATH).resolve()
if not BASE_DIR_PATH.is_dir():
    raise ValueError(f"Directory `{BASE_DIR_PATH}` does not exist.")

# Collect TDMS files from the specified directory path.
TDMS_PATHS = {}
for path in BASE_DIR_PATH.glob("*.tdms"):
    print(f"\tfound `{path.name}`")
    TDMS_PATHS[path.stem] = path

### Experiment viewer

In [ ]:
# Receive experiment name.
experiment_name = input("Experiment name: ")
if experiment_name not in TDMS_PATHS:
    raise ValueError(f"Experiment `{experiment_name}` not found.")
print(f"\tfound `{TDMS_PATHS[experiment_name]}`")

In [ ]:
# Load raw measurement from TDMS file.
print(f"[{datetime.datetime.now()}] Process experiment `{experiment_name}`")
print(f"[{datetime.datetime.now()}] \tload data")
current, meta = load_measurements(
    TDMS_PATHS[experiment_name],
    "PXIe-4309"
)

#### Coarse-grained view

In [ ]:
# Average and display coarse-grain average of current consumption.
print(f"[{datetime.datetime.now()}] \taverage current trace ({1/POWER_AVG_WINDOW_COARSE:.1f} Hz)")
current_avg_coarse = average_consumption(
    current,
    POWER_AVG_WINDOW_COARSE
)

output_notebook()
show(plot_power_signals(current_avg_coarse))

#### Fine-grained view

In [ ]:
# Get analysis boundaries from user.
time_start = input("Experiment start time: ")
time_start = float(time_start)
time_stop = input("Experiment stop time: ")
time_stop = float(time_stop)

print(f"[{datetime.datetime.now()}] \tanalysis range [{time_start}, {time_stop}]")
current_trimmed = current[time_start:time_stop]

In [ ]:
# Compute experiment power consumption.
print(f"[{datetime.datetime.now()}] \tcompute power consumption")
power, overall, normalized = compute_power_consumption(
    current_trimmed,
    POWER_LINE_PIN_VDD
)
for name in overall.index:
    print(f"{name:9} : {overall[name]:7.3f} W ({normalized[name] * 100:6.2f} %)")
print(f"=" * 35)
print(f"{'TOT':9} : {overall.sum():7.3f} W ({normalized.sum() * 100:6.2f} %)")

# Average experiment power consumption to display results.
print(f"[{datetime.datetime.now()}] \taverage power trace ({1/POWER_AVG_WINDOW_COARSE:.1f} Hz)")
power_avg_coarse = average_consumption(
    power,
    POWER_AVG_WINDOW_COARSE
)
print(f"[{datetime.datetime.now()}] \taverage power trace ({1/POWER_AVG_WINDOW_FINE:.1f} Hz)")
power_avg_fine = average_consumption(
    power,
    POWER_AVG_WINDOW_FINE
)
print(f"[{datetime.datetime.now()}] \tfinish.")

# Plot power traces.
output_notebook()

show(plot_power_signals(power_avg_fine))
show(plot_power_contributions_vbar(power_avg_coarse))
show(plot_power_contributions_varea(power_avg_fine))